In [3]:
import os, sys
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations

sys.path.insert(0,'../../LongTermEMG-master')

# Process Data

In [2]:
from LongTermClassificationMain.PrepareAndLoadDataLongTerm.prepare_dataset_utils import butter_bandpass_filter, \
    show_filtered_signal, load_timestamps_from_participant, get_angles_from_positions_3d_arm
from LongTermClassificationMain.PrepareAndLoadDataLongTerm import feature_extraction

In [3]:
os.chdir("/home/laiy/gitrepos/msr_final/LongTermEMG-master/LongTermClassificationMain/PrepareAndLoadDataLongTerm")
print(os.listdir("../../"))

['longterm_dataset_3DC.zip', 'longterm_dataset_3DC', 'README.md', 'LongTermClassificationMain', 'datasets', 'TransferLearning', '.idea', 'Weights_TSD']


In [4]:
list_participant_training_1_to_skip = ["Participant0/Training1", "Participant0/Evaluation2", "Participant0/Evaluation3",
                                       "Participant2/Training1", "Participant2/Evaluation2", "Participant2/Evaluation3"]

In [5]:
features_set_name = "TSD_features_set"
feature_set_function = feature_extraction.getTSD
path="../../datasets/longterm_dataset_3DC"
number_of_gestures=11
number_of_cycles=4
window_size=150 # 150ms frame with 100 overlap
size_non_overlap=50

In [6]:
def getTSDfeatures_for_one_representation(vector):
    """
    % Implementation from Rami Khusaba
    % Time-domain power spectral moments (TD-PSD)
    % Using Fourier relations between time domina and frequency domain to
    % extract power spectral moments dircetly from time domain.
    %
    % Modifications
    % 17/11/2013  RK: Spectral moments first created.
    % 02/03/2014  AT: I added 1 to the function name to differentiate it from other versions from Rami
    % 01/02/2016  RK: Modifed this code intosomewhat deep structure
    %
    % References
    % [1] A. Al-Timemy, R. N. Khushaba, G. Bugmann, and J. Escudero, "Improving the Performance Against Force Variation of EMG Controlled Multifunctional Upper-Limb Prostheses for Transradial Amputees",
    %     IEEE Transactions on Neural Systems and Rehabilitation Engineering, DOI: 10.1109/TNSRE.2015.2445634, 2015.
    % [2] R. N. Khushaba, Maen Takruri, Jaime Valls Miro, and Sarath Kodagoda, "Towards limb position invariant myoelectric pattern recognition using time-dependent spectral features",
    %     Neural Networks, vol. 55, pp. 42-58, 2014.
    """

    # Get the size of the input signal
    samples, channels = vector.shape

    if channels > samples:
        vector = np.transpose(vector)
        samples, channels = channels, samples

    # Root squared zero order moment normalized
    m0 = np.sqrt(np.nansum(vector ** 2, axis=0))[:, np.newaxis]
    m0 = m0 ** .1 / .1

    # Prepare derivatives for higher order moments
    d1 = np.diff(np.concatenate([np.zeros((1, channels)), vector], axis=0), n=1, axis=0)
    d2 = np.diff(np.concatenate([np.zeros((1, channels)), d1], axis=0), n=1, axis=0)

    # Root squared 2nd and 4th order moments normalized
    m2 = (np.sqrt(np.nansum(d1 ** 2, axis=0)) / (samples - 1))[:, np.newaxis]
    m2 = m2 ** .1 / .1

    m4 = (np.sqrt(np.nansum(d2 ** 2, axis=0)) / (samples - 1))[:, np.newaxis]
    m4 = m4 ** .1 / .1

    # Sparseness
    sparsi = m0 / np.sqrt(np.abs(np.multiply((m0 - m2) ** 2, (m0 - m4) ** 2)))

    # Irregularity Factor
    IRF = m2 / np.sqrt(np.multiply(m0, m4))

    # Coefficient of Variation
    COV = (np.nanstd(vector, axis=0, ddof=1) / np.nanmean(vector, axis=0))[:, np.newaxis]

    # Teager-Kaiser energy operator
    TEA = np.nansum(d1 ** 2 - np.multiply(vector[0:samples, :], d2), axis=0)[:, np.newaxis]

    # All features together
    STDD = np.nanstd(m0, axis=0, ddof=1)[:, np.newaxis]

    if channels > 2:
        Feat = np.concatenate((m0 / STDD, (m0 - m2) / STDD, (m0 - m4) / STDD, sparsi, IRF, COV, TEA), axis=0)
    else:
        Feat = np.concatenate((m0, m0 - m2, m0 - m4, sparsi, IRF, COV, TEA), axis=0)

    Feat = np.log(np.abs(Feat)).flatten()

    return Feat


In [7]:
def getTSD(all_channels_data_in_window):
    """
    This uses window emg directly instead of extracting a window then apply TSD like in Rami'code
    
        %Implementation from Rami Khusaba adapted to our code
        % References
        % [1] A. Al-Timemy, R. N. Khushaba, G. Bugmann, and J. Escudero, "Improving the Performance Against Force Variation of EMG Controlled Multifunctional Upper-Limb Prostheses for Transradial Amputees",
        %     IEEE Transactions on Neural Systems and Rehabilitation Engineering, DOI: 10.1109/TNSRE.2015.2445634, 2015.
        % [2] R. N. Khushaba, Maen Takruri, Jaime Valls Miro, and Sarath Kodagoda, "Towards limb position invariant myoelectric pattern recognition using time-dependent spectral features",
        %     Neural Networks, vol. 55, pp. 42-58, 2014.
    """
    # x should be a numpy array
    all_channels_data_in_window = np.swapaxes(np.array(all_channels_data_in_window), 1, 0)

    if len(all_channels_data_in_window.shape) == 1:
        all_channels_data_in_window = all_channels_data_in_window[:, np.newaxis]

    datasize = all_channels_data_in_window.shape[0]
    Nsignals = all_channels_data_in_window.shape[1]

    # prepare indices of each 2 channels combinations
    # NCC = Number of channels to combine
    NCC = 2
    Indx = np.array(list(combinations(range(Nsignals), NCC)))
    print("Idx = ", np.shape(Indx))
    print("Nsignals = ", Nsignals)

    # allocate memory
    # define the number of features per channel
    NFPC = 7

    # Preallocate memory
    feat = np.zeros((Indx.shape[0] * NFPC + Nsignals * NFPC))
    print("feat shape = ", feat.shape)

    # Step1.1: Extract between-channels features
    ebp = getTSDfeatures_for_one_representation(
        all_channels_data_in_window[:, Indx[:, 0]] - all_channels_data_in_window[:, Indx[:, 1]])
    efp = getTSDfeatures_for_one_representation(
        np.log(
            (all_channels_data_in_window[:, Indx[:, 0]] - all_channels_data_in_window[:, Indx[:, 1]]) ** 2 + np.spacing(
                1)) ** 2)
    # Step 1.2: Correlation analysis
    num = np.multiply(efp, ebp)
    den = np.sqrt(np.multiply(efp, efp)) + np.sqrt(np.multiply(ebp, ebp))
    feat[range(Indx.shape[0] * NFPC)] = num / den

    # Step2.1: Extract within-channels features
    ebp = getTSDfeatures_for_one_representation(all_channels_data_in_window)
    efp = getTSDfeatures_for_one_representation(np.log((all_channels_data_in_window) ** 2 + np.spacing(1)) ** 2)
    # Step2.2: Correlation analysis
    num = np.multiply(efp, ebp)
    den = np.sqrt(np.multiply(efp, efp)) + np.sqrt(np.multiply(ebp, ebp))
    feat[np.max(range(Indx.shape[0] * NFPC)) + 1:] = num / den
    return feat


In [8]:
def get_dataset_with_features_set(dataset):
    dataset_to_return = []
    for i, example in enumerate(dataset):
#         print(i, " example from dataset =  ", np.shape(example))
        example_formatted = []
        all_zero_channel = False
        for vector_electrode in example:
            # If only 0. the sensor was not recording correctly and we should ignore this example
            if np.sum(vector_electrode) == 0:
                all_zero_channel = True
        if all_zero_channel is False:
            example_formatted = getTSD(example)
            dataset_to_return.append(np.array(example_formatted).transpose().flatten())
    return dataset_to_return

In [9]:
def format_examples(emg_examples, feature_set_function, window_size=150, size_non_overlap=50):
    ''' 
    emg_examples: list of emg signals, each member represent one recording of a 10 channel emg
    
    feature_set_function:
        shape(formated_example) = (97, 10, 150)
        shape(featured_example) = (97, 385)
        
        Take each window emg from examples_to_calculate_features_set_from (97 windows total):
            NCC = 2  # NCC = Number of channels to combine
            NFPC = 7 # define the number of features per channel
            feat = np.zeros((Indx.shape[0] * NFPC + Nsignals * NFPC))
    
    window_size, size_non_verlap: 150ms frame with 100ms overlap    
        ignoring leftover examples at the end
        
    '''
    examples_to_calculate_features_set_from = []
    examples_to_calculate_features_set_from_2 = []
    example = []
    for emg_vector in emg_examples:
        if len(example) == 0:
            example = emg_vector
        else:
            example = np.row_stack((example, emg_vector))
        
        if len(example) >= window_size:
            # The example is of the shape TIME x CHANNEL. Make it of the shape CHANNEL x TIME
            example = example.transpose()
            # Go over each channel and bandpass filter it between 20 and 495 Hz.
            example_filtered = []
            for channel in example:
                channel_filtered = butter_bandpass_filter(channel, lowcut=20, highcut=495, fs=1000, order=4)
                # show_filtered_signal(channel, channel_filtered)
                example_filtered.append(channel_filtered)
            # Add the filtered example to the list of examples to return and transpose the example array again to go
            # back to TIME x CHANNEL
            examples_to_calculate_features_set_from.append(example_filtered)
             
            print("      @ format one example = ", np.shape(example_filtered))
            formated_example_filtered = getTSD(example_filtered)
            examples_to_calculate_features_set_from_2.append(np.array(formated_example_filtered).transpose().flatten())
            print("      @ featured one example = ", np.shape(examples_to_calculate_features_set_from_2))
            
            example = example.transpose()
            # Remove part of the data of the example according to the size_non_overlap variable
            example = example[size_non_overlap:]
            
            
    print("   @ formatting example = ", np.shape(examples_to_calculate_features_set_from))
    
#     examples_features_set_calculated = feature_extraction.get_dataset_with_features_set(
#         dataset=examples_to_calculate_features_set_from, features_set_function=feature_set_function)
    
    examples_features_set_calculated = get_dataset_with_features_set(dataset=examples_to_calculate_features_set_from)
    print("   @ featured example = ", np.shape(examples_features_set_calculated))
    
    print("CHECKING", np.allclose(examples_features_set_calculated, examples_to_calculate_features_set_from_2))

    return examples_features_set_calculated

In [10]:
from handcrafted_features_prepare_from_from_raw_dataset import get_highest_average_emg_window

def read_files_to_format_training_session(path_folder_examples, feature_set_function,
                                          number_of_cycles, number_of_gestures, window_size,
                                          size_non_overlap):
    '''
    when #cycle = 1, examples are recorded at highest activation
        
    '''
    examples_training, labels_training = [], []
    # Check if the folder is empty, if so, skip it
    # path_folder_examples = path + "/" + folder_participant + "/" + training_directory + "/EMG"
    if len(os.listdir(path_folder_examples)) == 0:
        return [], [], []
#     print(os.listdir(path_folder_examples))
#--------  filename = `3dc_EMG_gesture_<#cycle>_<#gesture>.txt`-------------
    highest_activation_per_gesture = []
    for cycle in range(number_of_cycles):
        # path_folder_examples = path + folder_participant + "/" + training_directory + "/EMG"
        # This one instance, the participant only recorded one cycle of training. Skip it
        for participant_session_to_skip in list_participant_training_1_to_skip:
            if participant_session_to_skip in path_folder_examples:
                return [], [], []
        path_emg = path_folder_examples + "/3dc_EMG_gesture_%d_" % cycle
        examples, labels = [], []
        for gesture_index in range(number_of_gestures):
            examples_to_format = []
            read_file = path_emg + '%d.txt' % gesture_index
#             print("READ FILE ", read_file)
            
            for line in open(read_file):
                #  strip() remove the "\n" character, split separate the data in a list. np.float
                #  transform each element of the list from a str to a float
                emg_signal = np.float32(line.strip().split(","))
                examples_to_format.append(emg_signal)
#             print("   example emg from file (to format)", np.shape(examples_to_format))
            
            if cycle == 1:  # This cycle is the second cycle and correspond to the highest effort baseline. Record it.
                if gesture_index == 0:
                    highest_activation_per_gesture.append(0)
                else:
                    highest_activation_per_gesture.append(get_highest_average_emg_window(
                        examples_to_format, window_for_moving_average=window_size))
                    
#             print("   start formating")
            examples_formatted = format_examples(examples_to_format,
                                                 feature_set_function=feature_set_function, window_size=window_size,
                                                 size_non_overlap=size_non_overlap)
            examples.extend(examples_formatted)
            labels.extend(np.ones(len(examples_formatted)) * gesture_index)
#             print("--- format complete for gesture ", gesture_index, " #examples = ", len(examples))
            
        print("   SHAPE SESSION ", cycle, " EXAMPLES: ", np.shape(examples))
        examples_training.append(examples)
        labels_training.append(labels)
        print("   SHAPE ALL SESSION EXAMPLES: ", np.shape(examples_training))

    return examples_training, labels_training, highest_activation_per_gesture

In [11]:
examples_training_sessions_datasets, labels_training_sessions_datasets = [], []
highest_activation_participants = []
examples_evaluation_sessions_datasets, labels_evaluation_sessions_datasets, timestamps_emg_evaluation = [], [], []
angles_with_timestamps_emg_evaluation = []

training_datetimes, evaluation_datetimes = [], []

### Each participant has four training sessions, each session has four cycles

In [12]:
for index_participant in range(1):
    folder_participant = "/Participant" + str(index_participant)
    sessions_directories = os.listdir(path + folder_participant)
    display("sessions_directories", sessions_directories)
    
#     Don't need datetime
#     training_datetime, evaluation_datetime = load_timestamps_from_participant(path + folder_participant)
#     training_datetimes.append(training_datetime)
#     display("training_datetime ", training_datetime)
#     evaluation_datetimes.append(evaluation_datetime)
#     display("evaluation_datetime ", evaluation_datetime)

    examples_participant_training_sessions, labels_participant_training_sessions = [], []
    highest_activation_per_session = []
    examples_participant_evaluation_sessions, labels_participant_evaluation_sessions = [], []
    timestamps_evaluation_participant_sessions, angles_with_timestamps_participant_sessions = [], []
    
    
    """
    Don't need Evaluation part for now, only process Training 
    """

    for session_directory in sessions_directories:
        print("current dir = ", session_directory)

        if "Training" in session_directory:
            path_folder_examples = path + "/" + folder_participant + "/" + session_directory + "/EMG"
            examples_training, labels_training, highest_activation_per_gesture = \
                read_files_to_format_training_session(path_folder_examples=path_folder_examples,
                                                      number_of_cycles=number_of_cycles,
                                                      number_of_gestures=number_of_gestures,
                                                      window_size=window_size,
                                                      size_non_overlap=size_non_overlap,
                                                      feature_set_function=feature_set_function)
            if len(examples_training) > 0:
                # These instances, the participant only recorded one cycle of training. Skip it
                skip_it = False
                for participant_session_to_skip in list_participant_training_1_to_skip:
                    if participant_session_to_skip in path_folder_examples:
                        skip_it = True
                if skip_it is False:
                    examples_participant_training_sessions.append(examples_training)
                    labels_participant_training_sessions.append(labels_training)
                    highest_activation_per_session.append(highest_activation_per_gesture)

                    print("@ traning sessions = ", np.shape(examples_participant_training_sessions))

'sessions_directories'

['Evaluation3',
 'Training3',
 'Training1',
 'Training5',
 'Evaluation8',
 'Evaluation0',
 'Evaluation6',
 'Evaluation7',
 'Training0',
 'Evaluation2',
 'Training4',
 'Evaluation9',
 'Training2',
 'Evaluation1',
 'Evaluation4',
 'Evaluation5']

current dir =  Evaluation3
current dir =  Training3
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (1, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (2, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (3, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (4, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (5, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (6, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (7, 385)
      @ format one examp

      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (79, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (80, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (81, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (82, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (83, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (84, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (85, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10

      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (10, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (11, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (12, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (13, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (14, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (15, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (16, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10

...

      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (82, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (83, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (84, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (85, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (86, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (87, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (88, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10

      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (19, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (20, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (21, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (22, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (23, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (24, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (25, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10

      @ featured one example =  (14, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (15, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (16, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (17, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (18, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (19, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (20, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (21, 385)
      @ format one example 

      @ featured one example =  (85, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (86, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (87, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (88, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (89, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)
      @ featured one example =  (90, 385)
      @ format one example =  (10, 150)
Idx =  (45, 2)
Nsignals =  10
feat shape =  (385,)


KeyboardInterrupt: 

In [ ]:
np.shape(labels_training_sessions_datasets[0][0][0])

In [ ]:
# store processed data to dictionary
print('traning examples ', np.shape(examples_participant_training_sessions))
examples_training_sessions_datasets = []
examples_training_sessions_datasets.append(examples_participant_training_sessions)
print('all traning examples ', np.shape(examples_training_sessions_datasets))

print('traning labels ', np.shape(labels_participant_training_sessions))
labels_training_sessions_datasets = []
labels_training_sessions_datasets.append(labels_participant_training_sessions)
print('all traning labels ', np.shape(labels_training_sessions_datasets))

dataset_dictionnary = {"examples_training": np.array(examples_training_sessions_datasets, dtype=object),
                       "labels_training": np.array(labels_training_sessions_datasets, dtype=object)}

In [ ]:
# store dictionary to pickle
training_session_dataset_dictionnary = {}
training_session_dataset_dictionnary["examples_training"] = dataset_dictionnary["examples_training"]
training_session_dataset_dictionnary["labels_training"] = dataset_dictionnary["labels_training"]

os.chdir("/home/laiy/gitrepos/msr_final/LongTermEMG-master/LongTermClassificationMain/PrepareAndLoadDataLongTerm")
with open("../Processed_datasets/%s_training_session.pickle" % features_set_name, 'wb') as f:
    pickle.dump(training_session_dataset_dictionnary, f, pickle.HIGHEST_PROTOCOL)

# Train TSD_DNN

In [ ]:
os.chdir("/home/laiy/gitrepos/msr_final/LongTermEMG-master/LongTermClassificationMain/TrainingsAndEvaluations/ForTrainingSessions/TSD_DNN")
print(os.listdir("../../"))
with open("../../../Processed_datasets/TSD_features_set_training_session.pickle", 'rb') as f:
    dataset_training = pickle.load(file=f)

examples_datasets_train = dataset_training['examples_training']
print('traning examples ', np.shape(examples_datasets_train))
labels_datasets_train = dataset_training['labels_training']
print('traning labels ', np.shape(labels_datasets_train))

In [18]:
a = [0,0]
1/np.mean(a, dtype='float')

ValueError: output parameter for reduction operation add has too many dimensions

In [7]:
np.array(list(combinations(range(8), 2)))   # (28,2)

array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [0, 5],
       [0, 6],
       [0, 7],
       [1, 2],
       [1, 3],
       [1, 4],
       [1, 5],
       [1, 6],
       [1, 7],
       [2, 3],
       [2, 4],
       [2, 5],
       [2, 6],
       [2, 7],
       [3, 4],
       [3, 5],
       [3, 6],
       [3, 7],
       [4, 5],
       [4, 6],
       [4, 7],
       [5, 6],
       [5, 7],
       [6, 7]])